<a href="https://colab.research.google.com/github/prince786838/repo/blob/main/important_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import numpy as np
import datetime as dt
import hmac
import json
import pytz
import time
import hashlib
import base64
import pandas as pd
import random
import math
import concurrent.futures
import threading
import sys

In [ ]:
market_detail_url = "https://api.coindcx.com/exchange/v1/markets_details"
detail_df = pd.DataFrame(requests.get(market_detail_url).json())
detail_df = detail_df[(detail_df['base_currency_short_name']=='INR') | (detail_df['base_currency_short_name']=='USDT')].reset_index(drop=True)

market_details = {detail_df['coindcx_name'][i]:{'base_currency_precision':detail_df['base_currency_precision'][i],
                                          'target_currency_precision':detail_df['target_currency_precision'][i],
                                           'min_qty' : detail_df['min_quantity'][i],
                                           'max_qty' : detail_df['max_quantity'][i]
                                           } for i in range(len(detail_df))}

In [ ]:
def scan_orderbook(pair, order_type='BIDS'):
    '''
    Retrieve the order book for a specific cryptocurrency trading pair on Coindcx.

    Parameters:
    - pair (str): Cryptocurrency trading pair, e.g., 'BTC'.
    - order_type (str, optional): Type of order book to retrieve, 'BIDS' or 'ASKS'. Defaults to 'BIDS'.

    Returns:
    - orderbook_df (pd.DataFrame): DataFrame containing the order book data.
    '''

    # URL for fetching the order book data
    trade_url = "https://public.coindcx.com/market_data/orderbook?pair=I-{}_INR".format(pair)

    try:
        # Make the request to fetch the order book data
        response = requests.get(trade_url)
        response.raise_for_status()  # Raise an exception for unsuccessful requests
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        exit()

    # Convert the JSON response to a Pandas DataFrame
    data = pd.DataFrame(response.json())
    data.reset_index(drop=False, inplace=True)
    data.rename(columns={'index': 'price'}, inplace=True)

    # Process the order book based on the specified order type
    if order_type == 'BIDS':
        COIN_INR_BIDS = data[data['asks'].isnull()].reset_index(drop=True).drop('asks', axis=1)
        COIN_INR_BIDS.rename(columns={'bids': 'qty'}, inplace=True)
        COIN_INR_BIDS = COIN_INR_BIDS.astype({'price': 'float', 'qty': 'float'})
        COIN_INR_BIDS['value'] = COIN_INR_BIDS['qty'] * COIN_INR_BIDS['price']
        return COIN_INR_BIDS
    elif order_type == 'ASKS':
        COIN_INR_ASKS = data[data['bids'].isnull()].reset_index(drop=True).drop('bids', axis=1)
        COIN_INR_ASKS.rename(columns={'asks': 'qty'}, inplace=True)
        COIN_INR_ASKS = COIN_INR_ASKS.astype({'price': 'float', 'qty': 'float'})
        COIN_INR_ASKS['value'] = COIN_INR_ASKS['qty'] * COIN_INR_ASKS['price']
        return COIN_INR_ASKS
    else:
        print("Invalid order_type. Use 'BIDS' or 'ASKS'.")
        return None

In [ ]:
def check_balance(coin, locked=False):
    '''
    Check the balance of a specific cryptocurrency in the Coindcx exchange.

    Parameters:
    - coin (str): Cryptocurrency symbol, e.g., 'BTC'.
    - locked (bool, optional): Whether to include locked balances. Defaults to False.

    Returns:
    - balance (float or None): The balance of the specified cryptocurrency.
                              If the currency is not found, returns None.
    '''

    # Get the current timestamp
    timeStamp = int(round(time.time() * 1000))

    # Create the request body
    body = {"timestamp": timeStamp}
    json_body = json.dumps(body, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for checking balances
    url = "https://api.coindcx.com/exchange/v1/users/balances"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to check balances
    response = requests.post(url, data=json_body, headers=headers)

    # Process the response based on the 'locked' parameter
    if not locked:
        for e in response.json():
            if e['currency'] == coin:
                return float(e['balance'])
    else:
        for e in response.json():
            if e['currency'] == coin:
                return float(e['balance']) + float(e['locked_balance'])

    # Return None if the currency is not found
    return None

In [ ]:
def create_order(pair, price, order_qty, base_currency_precision, timestamp, side='buy', order_type='limit_order'):
    '''
    Place a cryptocurrency order on the Coindcx exchange.

    Parameters:
    - pair (str): Trading pair, e.g., 'BTC'.
    - price (float): Price per unit for a limit order.
    - order_qty (float): Total quantity of the order.
    - base_currency_precision (int): Precision for rounding the price.
    - timestamp (int): Current timestamp for the request.
    - side (str, optional): Order side, 'buy' or 'sell'. Defaults to 'buy'.
    - order_type (str, optional): Type of order, 'market_order' or 'limit_order'. Defaults to 'limit_order'.

    Returns:
    - order_id (str): ID of the created order.
    '''

    # Construct the order body
    body = {
        "side": side,  # Toggle between 'buy' or 'sell'.
        "order_type": order_type,  # Toggle between a 'market_order' or 'limit_order'.
        "market": f"{pair}INR",
        "price_per_unit": round(price, base_currency_precision),  # This parameter is only required for a 'limit_order'
        "total_quantity": order_qty,
        "timestamp": timestamp
    }

    # Convert the order body to JSON
    json_body = json.dumps(body, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for placing orders
    order_url = "https://api.coindcx.com/exchange/v1/orders/create"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to place the order
    response = requests.post(order_url, data=json_body, headers=headers)

    # Extract the order ID from the response
    order_id = response.json()['orders'][0]['id']

    # Return the order ID
    return order_id

In [ ]:
def edit_order(order_id, new_price):
    '''
    Edit an existing order by modifying its price.

    Parameters:
    - order_id (str): The ID of the order to be edited.
    - new_price (float): The new price for the order.

    Returns:
    - data (dict): Response data from the API after editing the order.
    '''

    # Generating a timestamp.
    timeStamp = int(round(time.time() * 1000))

    # Construct the request body
    body = {
        "id": order_id,          # Enter your Order ID here.
        "timestamp": timeStamp,
        "price_per_unit": new_price  # Enter the new price here.
    }

    # Convert the request body to JSON
    json_body = json.dumps(body, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for editing orders
    url = "https://api.coindcx.com/exchange/v1/orders/edit"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to edit the order
    response = requests.post(url, data=json_body, headers=headers)

    # Extract and return the response data
    data = response.json()
    return data

In [ ]:
def cancel_order(order_id, MAX_RETRY_ATTEMPTS=3, RETRY_DELAY_SECONDS=1):
    '''
    Cancel an existing order on the Coindcx exchange.

    Parameters:
    - order_id (str): The ID of the order to be canceled.
    - MAX_RETRY_ATTEMPTS (int, optional): Maximum number of retry attempts. Defaults to 3.
    - RETRY_DELAY_SECONDS (int, optional): Delay in seconds between retry attempts. Defaults to 1.

    Returns:
    - response_data (dict or None): Response data from the API after canceling the order.
                                    Returns None if the order cannot be canceled after max retry attempts.
    '''

    retry_count = 0

    while retry_count < MAX_RETRY_ATTEMPTS:
        try:
            # Generating a timestamp.
            timeStamp = int(round(time.time() * 1000))

            # Construct the request body
            json_body = json.dumps({"id": order_id, "timestamp": timeStamp}, separators=(',', ':'))

            # Generate the request signature
            signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

            # API endpoint for canceling orders
            url = "https://api.coindcx.com/exchange/v1/orders/cancel"

            # HTTP headers for the request
            headers = {
                'Content-Type': 'application/json',
                'X-AUTH-APIKEY': key,
                'X-AUTH-SIGNATURE': signature
            }

            # Make the API request to cancel the order
            response = requests.post(url, data=json_body, headers=headers)

            # Check the response status code
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to cancel order. Status code: {response.status_code}")
                retry_count += 1
                time.sleep(RETRY_DELAY_SECONDS)
        except Exception as e:
            print(f"Error during order cancellation: {str(e)}")
            retry_count += 1
            time.sleep(RETRY_DELAY_SECONDS)

    print(f"Max retry attempts reached. Failed to cancel order with ID: {order_id}")
    return None


In [ ]:
def check_order_status(order_id):
    '''
    Check the status of an existing order on the Coindcx exchange.

    Parameters:
    - order_id (str): The ID of the order for which the status is to be checked.

    Returns:
    - status_dict (dict): Dictionary containing the status information of the specified order.
    '''

    # Generating a timestamp.
    timeStamp = int(round(time.time() * 1000))

    # Construct the request body
    json_body = json.dumps({"id": order_id, "timestamp": timeStamp}, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for checking order status
    status_url = "https://api.coindcx.com/exchange/v1/orders/status"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to check order status
    status_dict = requests.post(status_url, data=json_body, headers=headers).json()
    return status_dict


In [ ]:
def get_max_buy_price(pair,margin=2,ignore_value=500):
    """
    Calculate the maximum buy price for a given trading pair.

    :pair: The trading pair for which to calculate the max buy price.
    :margin: The margin percentage to be used in the calculation. Default is 2%.
    :ignore_value: The threshold value to ignore in the order book. Default is 500.

    :return: The calculated maximum buy price.
    """
    temp_df = scan_orderbook(pair,'ASKS')
    df = temp_df[temp_df['value']>ignore_value]
    sell_price = df['price'].iloc[0]
    max_buy_price = sell_price*(1-margin/100)
    return max_buy_price

In [ ]:
def get_min_sell_price(pair,margin=2,ignore_value=500):
    """
    Calculate the minimum sell price for a given trading pair.

    :pair: The trading pair for which to calculate the max buy price.
    :margin: The margin percentage to be used in the calculation. Default is 2%.
    :ignore_value: The threshold value to ignore in the order book. Default is 500.

    :return: The calculated minimum sell price.
    """
    temp_df = scan_orderbook(pair,'BIDS')
    df = temp_df[temp_df['value']>ignore_value]
    buy_price = df['price'].iloc[0]
    min_sell_price = buy_price*(1+margin/100)
    return min_sell_price

In [ ]:
def get_coin_value(pair, locked=True):
    '''
    Calculate the value of a specific cryptocurrency in the Coindcx exchange.

    Parameters:
    - pair (str): Cryptocurrency symbol, e.g., 'BTC'.
    - locked (bool, optional): Whether to include locked balances. Defaults to True.

    Returns:
    - coin_value (float): The calculated value of the specified cryptocurrency.
    '''

    # Get the balance of the specified cryptocurrency
    coin_balance = check_balance(pair, locked)

    # If balance is None, set it to 0
    coin_balance = coin_balance if coin_balance else 0

    # Scan the order book to get the current price
    bids = scan_orderbook(pair)
    price = bids['price'].values[0]

    # Calculate the value of the cryptocurrency
    coin_value = coin_balance * price

    return coin_value

In [ ]:
def get_active_orders(pair, side='buy', market='INR'):
    '''
    Retrieve the active orders for a specific cryptocurrency trading pair on the Coindcx exchange.

    Parameters:
    - pair (str): Cryptocurrency symbol, e.g., 'BTC'.
    - side (str, optional): Order side, 'buy' or 'sell'. Defaults to 'buy'.
    - market (str, optional): Market for the order, e.g., 'INR'. Defaults to 'INR'.

    Returns:
    - orders_df (pd.DataFrame): DataFrame containing the active orders for the specified pair.
    '''

    # Generating a timestamp.
    timeStamp = int(round(time.time() * 1000))

    # Construct the request body
    body = {
        "side": side,                             # Toggle between a 'buy' or 'sell' order.
        "market": f"{pair}{market}",              # Replace 'SNTBTC' with your desired market pair.
        "timestamp": timeStamp
    }

    # Convert the request body to JSON
    json_body = json.dumps(body, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for retrieving active orders
    url = "https://api.coindcx.com/exchange/v1/orders/active_orders"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to retrieve active orders
    response = requests.post(url, data=json_body, headers=headers)
    df = response.json()

    # Extract the list from the 'orders' key
    orders_list = df['orders']

    # Create a DataFrame from the list
    orders_df = pd.DataFrame(orders_list)

    return orders_df

In [ ]:
def convert_epoch_to_datetime(epoch_timestamp, timezone='Asia/Kolkata'):
    # Convert epoch timestamp to a datetime object
    datetime_obj = pd.to_datetime(epoch_timestamp, unit='ms', origin='unix')

    # Apply the desired timezone
    datetime_obj = datetime_obj.tz_localize('UTC').tz_convert(timezone)

    # Extract date, hour, minute, second, and Unix timestamp
    date = datetime_obj.strftime('%Y-%m-%d')
    hour = datetime_obj.hour
    minute = datetime_obj.minute
    second = datetime_obj.second
    unix_timestamp = datetime_obj.timestamp()  # Unix timestamp in seconds

    return date, hour, minute, second

In [ ]:
def trade_history(pair, lookback_hours=24*365):
    """
    Calculate the net INR value of buy & sell orders for a given trading pair within a specified lookback period.

    Parameters:
    - pair (str): The trading pair for which to calculate the net buy value (buy-sell).
    - lookback_hours (int, optional): The lookback period in hours to consider for trade history.
                                      Default is one year (24 hours * 365 days).

    Returns:
    - data (pd.DataFrame): DataFrame containing the trade history data.
                           If there's no trade data available, returns 0.

    Note: This function assumes that variables 'secret_bytes' and 'key' are defined elsewhere in your code,
    as they are used for authentication with the API.
    """

    timeStamp = int(round(time.time() * 1000))
    from_timestamp = timeStamp - 3600 * 1000 * lookback_hours

    # Construct the request body
    body = {
        "limit": 5000,
        "timestamp": timeStamp,
        "sort": "desc",
        "from_timestamp": from_timestamp,
        "to_timestamp": timeStamp,
        "symbol": f"{pair}INR"
    }

    # Convert the request body to JSON
    json_body = json.dumps(body, separators=(',', ':'))

    # Generate the request signature
    signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

    # API endpoint for retrieving trade history
    url = "https://api.coindcx.com/exchange/v1/orders/trade_history"

    # HTTP headers for the request
    headers = {
        'Content-Type': 'application/json',
        'X-AUTH-APIKEY': key,
        'X-AUTH-SIGNATURE': signature
    }

    # Make the API request to retrieve trade history
    response = requests.post(url, data=json_body, headers=headers)
    data = pd.DataFrame(response.json())

    # Check if the DataFrame is empty
    if data.empty:
        return 0
    else:
        # Convert relevant columns to the appropriate data types
        data['price'] = data['price'].astype(float)
        data['quantity'] = data['quantity'].astype(float)
        data['fee_amount'] = data['fee_amount'].astype(float)
        data['side'] = data['side'].astype(str)

        # Calculate additional columns
        data['trade_val'] = data['price'] * data['quantity']
        data['date'], data['hour'], data['minute'], data['second'] = zip(*data['timestamp'].apply(lambda x: convert_epoch_to_datetime(x)))
        data['tds_paid'] = np.where(data['side'] == 'buy', 0, data['trade_val'] * 0.01)
        data['net_order_value'] = np.where(data['side'] == 'buy', -1 * (data['trade_val'] + data['fee_amount']), data['trade_val'] - data['fee_amount'] - data['tds_paid'])

        # Calculate cumulative net order value
        cumsum_reverse = data['net_order_value'][::-1].cumsum()[::-1]
        data['cum_net_order_value'] = cumsum_reverse

        # Calculate summary values
        net_sell = data['net_order_value'].sum()
        net_tds_paid = data['tds_paid'].sum()
        total_order_value = data['trade_val'].sum()
        total_fee_paid = data['fee_amount'].sum()

        print('total order value:', total_order_value)
        print('net sell:', net_sell)
        print('total_tds_paid:', net_tds_paid)
        print('total_fee_paid:', total_fee_paid)

        # Get the current coin value
        coin_value = get_coin_value(pair)
        print('coin value:', coin_value)

        return data

In [ ]:
import math

def round_down(value, decimals):
    '''
    Round down a given value to a specified number of decimals.

    Parameters:
    - value (float): The value to be rounded down.
    - decimals (int): The number of decimal places to round down to.

    Returns:
    - rounded_value (float): The rounded down value.
    '''
    factor = 10 ** decimals
    return math.floor(value * factor) / factor

In [ ]:
def cancel_all(pair, side='buy', market='INR'):
    '''
    Cancel orders for a specific trading pair on the Coindcx exchange.

    Parameters:
    - pair (str): Cryptocurrency symbol, e.g., 'BTC'.
    - side (str, optional): Order side, 'buy', 'sell', or 'all'. Defaults to 'buy'.
    - market (str, optional): Market for the order, e.g., 'INR'. Defaults to 'INR'.

    Returns:
    - response_data (dict): Response data from the API after canceling orders.
    '''

    # Generating a timestamp.
    timeStamp = int(round(time.time() * 1000))

    # Determine the side for the order
    if side == 'all':
        # If side is 'all', cancel both buy and sell orders
        sides_to_cancel = ['buy', 'sell']
    else:
        # Otherwise, cancel the specified side
        sides_to_cancel = [side]

    # List to store response data for each canceled side
    response_data_list = []

    for current_side in sides_to_cancel:
        # Construct the request body for the current side
        body = {
            "side": current_side,
            "market": f"{pair}{market}",
            "timestamp": timeStamp
        }

        # Convert the request body to JSON
        json_body = json.dumps(body, separators=(',', ':'))

        # Generate the request signature
        signature = hmac.new(secret_bytes, json_body.encode(), hashlib.sha256).hexdigest()

        # API endpoint for canceling orders
        url = "https://api.coindcx.com/exchange/v1/orders/cancel_all"

        # HTTP headers for the request
        headers = {
            'Content-Type': 'application/json',
            'X-AUTH-APIKEY': key,
            'X-AUTH-SIGNATURE': signature
        }

        # Make the API request to cancel orders for the current side
        response = requests.post(url, data=json_body, headers=headers)
        response_data = response.json()

        # Append the response data to the list
        response_data_list.append(response_data)

    # Return the response data list
    return response_data_list

In [2]:
def sell_whole(pair, market='INR'):
    '''
    Sell the entire coin holding at the top bid available on the Coindcx exchange.

    Parameters:
    - pair (str): Cryptocurrency symbol, e.g., 'BTC'.
    - market (str, optional): Market for the order, e.g., 'INR'. Defaults to 'INR'.

    Note: This function assumes that variables 'secret_bytes', 'key', 'market_details', 'cancel_all', 'check_balance', 'scan_orderbook',
    'round_down', 'create_order', and 'check_order_status' are defined elsewhere in your code,
    as they are used for various functionalities.

    The function cancels all open orders, then continuously checks the order book for the top bid,
    calculates the value of the entire coin holding, and places a sell order at the best bid price.
    It waits for the order to be filled and repeats the process until the entire coin holding is sold.
    '''

    # Cancel all open orders before selling
    cancel_all(pair, side='all', market=market)

    # Get precision details for the trading pair
    base_currency_precision = market_details[f"{pair}INR"]['base_currency_precision']
    target_currency_precision = market_details[f"{pair}INR"]['target_currency_precision']

    timestamp = int(round(time.time()*1000))

    while True:
        # Get the current coin balance
        coin_balance = check_balance(pair)
        coin_balance = coin_balance if coin_balance else 0

        # Scan the order book to get the top bid price
        bids = scan_orderbook(pair)
        best_bid_price = bids['price'].values[0]
        coin_value = coin_balance * best_bid_price

        if coin_value > 100:
            # We have sufficient amount to sell

            # Round down the order quantity to the target currency precision
            order_qty = round_down(coin_balance, target_currency_precision)

            # Place a sell order at the best bid price
            order_id = create_order(pair, price=best_bid_price, order_qty=order_qty,
                                    base_currency_precision=base_currency_precision,
                                    timestamp=timestamp, side='sell', order_type='limit_order')

            # Wait for 3 seconds
            time.sleep(3)

            # Check the order status
            status_dict = check_order_status(order_id)
            order_status = status_dict['status']

            if order_status == 'filled':
                # The order is fully filled, we sold the whole coin balance
                break
            else:
                # Order didn't fulfill, cancel the order
                cancel_order(order_id)
        else:
            # Sold the entire coin holding
            break